In [1]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import json
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Set up visualization style
sns.set(style="whitegrid")
plt.figure(figsize=(12, 6))


def load_ais_data(file_path):

    df = pd.read_csv(file_path, header=None, names=['id', 'timestamp', 'data'])

    # Parse the JSON data in the 'data' column
    df['data'] = df['data'].apply(json.loads)

    # Expand the JSON data into separate columns
    data_df = pd.json_normalize(df['data'])

    # Combine with the original DataFrame
    df = pd.concat([df.drop('data', axis=1), data_df], axis=1)

    # Convert timestamp to datetime
    df['timestamp'] = pd.to_datetime(df['timestamp'])

    return df

# Load the data
file_path = 'rhein.csv'
ais_data = load_ais_data(file_path)

print(ais_data.head())
ais_data.count()


        id                  timestamp  msg_type  repeat       mmsi  status  \
0  3700533 2025-03-08 10:26:16.403407         1       0  211572680     0.0   
1  3700534 2025-03-08 10:26:17.001696         1       0  211478290     0.0   
2  3700535 2025-03-08 10:26:17.432819         1       0  211668170    15.0   
3  3700536 2025-03-08 10:26:17.513798         1       0  244660836     0.0   
4  3700537 2025-03-08 10:26:19.133889         1       0  211668170    15.0   

    turn  speed accuracy       lon  ...  offset1_2  spare_3  type2_1  \
0 -128.0    3.2     True  8.315200  ...        NaN      NaN      NaN   
1    0.0    0.1     True  8.307820  ...        NaN      NaN      NaN   
2 -128.0   18.2    False  8.280108  ...        NaN      NaN      NaN   
3 -128.0    4.4     True  8.324817  ...        NaN      NaN      NaN   
4 -128.0   18.5    False  8.280207  ...        NaN      NaN      NaN   

   offset2_1  spare_4 year partno  vendorid  model  serial  
0        NaN      NaN  NaN    NaN    

id           150000
timestamp    150000
msg_type     150000
repeat       150000
mmsi         150000
              ...  
year             24
partno            6
vendorid          6
model             6
serial            6
Length: 67, dtype: int64

<Figure size 1200x600 with 0 Axes>

In [8]:
# Prepare the dataset
features = ['lat', 'lon', 'velocity_x', 'velocity_y', 'gradient_info']
target = ['lat', 'lon', 'velocity_x', 'velocity_y', 'gradient_info']

# Group by MMSI to calculate velocity components for each vessel separately
ais_data['velocity_x'] = ais_data.groupby('mmsi')['lon'].diff() / ais_data.groupby('mmsi')['timestamp'].diff().dt.total_seconds()
ais_data['velocity_y'] = ais_data.groupby('mmsi')['lat'].diff() / ais_data.groupby('mmsi')['timestamp'].diff().dt.total_seconds()

# Calculate the gradient (direction of movement)
ais_data['gradient_info'] = np.arctan2(ais_data['velocity_y'], ais_data['velocity_x'])  # Gradient as angle (direction)

# Display the updated data with velocity and gradient columns
ais_data[['mmsi', 'timestamp', 'lat', 'lon', 'velocity_x', 'velocity_y', 'gradient_info']].head(10)

# print(ais_data['velocity_x'].head(10))
# print(ais_data['velocity_y'].head(10))
# print(ais_data['velocity_x'].head(10))

,mmsi,timestamp,lat,lon,velocity_x,velocity_y,gradient_info
0,211572680,2025-03-08 10:26:16.403407,49.015880,8.315200,NaN,NaN,NaN
1,211478290,2025-03-08 10:26:17.001696,49.016428,8.307820,NaN,NaN,NaN
2,211668170,2025-03-08 10:26:17.432819,48.991250,8.280108,NaN,NaN,NaN
3,244660836,2025-03-08 10:26:17.513798,49.069933,8.324817,NaN,NaN,NaN
4,211668170,2025-03-08 10:26:19.133889,48.991307,8.280207,0.000058,0.000034,0.522403
5,205437990,2025-03-08 10:26:20.928263,49.020410,8.300710,NaN,NaN,NaN
6,211668170,2025-03-08 10:26:21.099798,48.991423,8.280402,0.000099,0.000059,0.536640
7,269057351,2025-03-08 10:26:23.155308,49.000200,8.288200,NaN,NaN,NaN
8,211668170,2025-03-08 10:26:23.312854,48.991538,8.280600,0.000089,0.000052,0.526188
9,211632780,2025-03-08 10:26:23.488851,48.978873,8.255463,NaN,NaN,NaN


In [10]:
# Choose a specific MMSI ID (e.g., 211572680)
mmsi_id = 211572680

# Filter the dataset for that specific MMSI ID
filtered_data = ais_data[(ais_data['mmsi'] == mmsi_id) & (ais_data['msg_type'] == 1)]

# Display the first 10 rows for that MMSI
filtered_data[['mmsi', 'timestamp', 'lat', 'lon', 'velocity_x', 'velocity_y', 'gradient_info']]


,mmsi,timestamp,lat,lon,velocity_x,velocity_y,gradient_info
0,211572680,2025-03-08 10:26:16.403407,49.015880,8.315200,NaN,NaN,NaN
12,211572680,2025-03-08 10:26:26.593755,49.015872,8.315450,0.000025,-7.850566e-07,-0.031989
26,211572680,2025-03-08 10:26:37.222831,49.015865,8.315687,0.000022,-6.585709e-07,-0.029527
37,211572680,2025-03-08 10:26:47.210330,49.015858,8.315927,0.000024,-7.008762e-07,-0.029158
49,211572680,2025-03-08 10:26:56.765464,49.015853,8.316175,0.000026,-5.232789e-07,-0.020159
...,...,...,...,...,...,...,...
146326,211572680,2025-03-11 02:23:31.749434,48.961820,8.211675,-0.000022,-3.600358e-06,-2.979091
146349,211572680,2025-03-11 02:24:21.749037,48.961642,8.210543,-0.000023,-3.560028e-06,-2.985626
146424,211572680,2025-03-11 02:27:13.009443,48.961037,8.206782,-0.000022,-3.532632e-06,-2.982098
146730,211572680,2025-03-11 02:39:32.608862,48.955032,8.193152,-0.000018,-8.119260e-06,-2.726606


In [17]:
# Normalize the features and target using MinMaxScaler
cols = list(dict.fromkeys(features + target))  # ['lat','lon','velocity_x','velocity_y','gradient_info']

# drop any rows where these five columns are NaN

clean = ais_data.dropna(subset=cols)
# 2. Scale only the unique columns
scaler = MinMaxScaler()
scaled_data = clean[cols].copy()
scaled_data[cols] = scaler.fit_transform(scaled_data[cols])

In [18]:
# 3. Prepare the dataset for training: Sequence data for LSTM
def create_sequences(data, seq_length=10):
    sequences = []
    targets   = []
    for i in range(len(data) - seq_length):
        # inputs: 10×5 features
        sequences.append(data.iloc[i:i+seq_length][features].values)
        # outputs: 5 dims
        targets.append(data.iloc[i+seq_length][target].values)
    return np.array(sequences), np.array(targets)

# 4. Create sequences for training
X, y = create_sequences(scaled_data, seq_length=10)

# 5. Convert to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

# 6. Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X_tensor, y_tensor, test_size=0.2, random_state=42
)

In [20]:

# Define the LSTM model in PyTorch
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        lstm_out, (h_n, c_n) = self.lstm(x)
        out = self.fc(lstm_out[:, -1, :])  # Take the last output for prediction
        return out

# Set parameters for the model
input_size = len(features)
hidden_size = 128
output_size = len(target)  # 5 outputs: lat, lon, velocity_x, velocity_y, gradient_info

# Instantiate the model
model = LSTMModel(input_size, hidden_size, output_size)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training the model
num_epochs = 100
batch_size = 128

for epoch in range(num_epochs):
    model.train()
    permutation = torch.randperm(X_train.size(0))

    for i in range(0, X_train.size(0), batch_size):
        indices = permutation[i:i + batch_size]
        batch_x, batch_y = X_train[indices], y_train[indices]

        # Zero gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(batch_x)

        # Compute the loss
        loss = criterion(outputs, batch_y)

        # Backward pass
        loss.backward()

        # Optimize the weights
        optimizer.step()

    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}")

# Evaluate the model
model.eval()
with torch.no_grad():
    y_pred = model(X_test)

# Print evaluation results
y_pred = y_pred.numpy()
y_test = y_test.numpy()


# Show some sample predictions vs actuals
for i in range(5):
    print(f"Prediction: {y_pred[i]}, Actual: {y_test[i]}")


Epoch [1/100], Loss: 0.0161
Epoch [2/100], Loss: 0.0134
Epoch [3/100], Loss: 0.0153
Epoch [4/100], Loss: 0.0160
Epoch [5/100], Loss: 0.0210
Epoch [6/100], Loss: 0.0133
Epoch [7/100], Loss: 0.0160
Epoch [8/100], Loss: 0.0165
Epoch [9/100], Loss: 0.0159
Epoch [10/100], Loss: 0.0133
Epoch [11/100], Loss: 0.0187
Epoch [12/100], Loss: 0.0145
Epoch [13/100], Loss: 0.0169
Epoch [14/100], Loss: 0.0125
Epoch [15/100], Loss: 0.0149
Epoch [16/100], Loss: 0.0136
Epoch [17/100], Loss: 0.0185
Epoch [18/100], Loss: 0.0163
Epoch [19/100], Loss: 0.0166
Epoch [20/100], Loss: 0.0147
Epoch [21/100], Loss: 0.0156
Epoch [22/100], Loss: 0.0223
Epoch [23/100], Loss: 0.0126
Epoch [24/100], Loss: 0.0184
Epoch [25/100], Loss: 0.0181
Epoch [26/100], Loss: 0.0195
Epoch [27/100], Loss: 0.0128
Epoch [28/100], Loss: 0.0132
Epoch [29/100], Loss: 0.0136
Epoch [30/100], Loss: 0.0156
Epoch [31/100], Loss: 0.0206
Epoch [32/100], Loss: 0.0144
Epoch [33/100], Loss: 0.0193
Epoch [34/100], Loss: 0.0150
Epoch [35/100], Loss: 0

AttributeError: 'numpy.ndarray' object has no attribute 'numpy'